In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, avg, count

In [0]:
# Initialize SparkSession
spark = SparkSession.builder.appName("SalesDataTransformation").getOrCreate()

In [0]:
#service_credential = "p-J8Q~C6Fc8tFwtS-QAE-qFP9EPuXE4SV5McAc5Q"
#application_id = "552a69ea-ab44-4842-8a03-7bcc62f38926"
#directory_id = "b99772b5-42c9-4cf6-829d-232ec2fbe8e7"

In [0]:
%python
#Set the storage account key
spark.conf.set(
    "fs.azure.account.key.dpstorage21.dfs.core.windows.net","o1zhSIwsiONj8+NSuytF22pskSpGLY4gKUIYWTkpNoGUOJrmfVGdIvdzDYByuwtBmtMtyvm0OB39+AStJwZKlQ==")

# List the files in the specified directory
display(dbutils.fs.ls("abfss://processed@dpstorage21.dfs.core.windows.net/sales_data_cleaned.csv"))

path,name,size,modificationTime
abfss://processed@dpstorage21.dfs.core.windows.net/sales_data_cleaned.csv/_SUCCESS,_SUCCESS,0,1733365413000
abfss://processed@dpstorage21.dfs.core.windows.net/sales_data_cleaned.csv/_committed_5790219034973124292,_committed_5790219034973124292,746,1733365413000
abfss://processed@dpstorage21.dfs.core.windows.net/sales_data_cleaned.csv/_committed_9102755207785539120,_committed_9102755207785539120,380,1733365356000
abfss://processed@dpstorage21.dfs.core.windows.net/sales_data_cleaned.csv/_started_5790219034973124292,_started_5790219034973124292,0,1733365413000
abfss://processed@dpstorage21.dfs.core.windows.net/sales_data_cleaned.csv/_started_9102755207785539120,_started_9102755207785539120,0,1733365355000
abfss://processed@dpstorage21.dfs.core.windows.net/sales_data_cleaned.csv/part-00000-tid-5790219034973124292-2dc58480-691d-4ad6-bec4-dc8f089c75c1-235-1-c000.csv,part-00000-tid-5790219034973124292-2dc58480-691d-4ad6-bec4-dc8f089c75c1-235-1-c000.csv,146330,1733365413000
abfss://processed@dpstorage21.dfs.core.windows.net/sales_data_cleaned.csv/part-00001-tid-5790219034973124292-2dc58480-691d-4ad6-bec4-dc8f089c75c1-236-1-c000.csv,part-00001-tid-5790219034973124292-2dc58480-691d-4ad6-bec4-dc8f089c75c1-236-1-c000.csv,146203,1733365413000
abfss://processed@dpstorage21.dfs.core.windows.net/sales_data_cleaned.csv/part-00002-tid-5790219034973124292-2dc58480-691d-4ad6-bec4-dc8f089c75c1-237-1-c000.csv,part-00002-tid-5790219034973124292-2dc58480-691d-4ad6-bec4-dc8f089c75c1-237-1-c000.csv,144539,1733365413000
abfss://processed@dpstorage21.dfs.core.windows.net/sales_data_cleaned.csv/part-00003-tid-5790219034973124292-2dc58480-691d-4ad6-bec4-dc8f089c75c1-238-1-c000.csv,part-00003-tid-5790219034973124292-2dc58480-691d-4ad6-bec4-dc8f089c75c1-238-1-c000.csv,144377,1733365413000


In [0]:
file_path = "abfss://processed@dpstorage21.dfs.core.windows.net/sales_data_cleaned.csv"
sales_data_df = spark.read.csv(file_path, header=True, inferSchema=True)

In [0]:
# 1. GroupBy Transformation: Summarize Sales by Product Line
sales_summary = (
    sales_data_df.groupBy("PRODUCTLINE")
    .agg(
        sum("SALES").alias("TOTAL_SALES"),
        avg("SALES").alias("AVERAGE_SALES"),
        count("SALES").alias("SALES_COUNT")
    )
)

In [0]:
# 1. GroupBy Transformation: Summarize Sales by Product Line
sales_summary = (
    sales_data_df.groupBy("PRODUCTLINE")
    .agg(
        sum("SALES").alias("TOTAL_SALES"),
        avg("SALES").alias("AVERAGE_SALES"),
        count("SALES").alias("SALES_COUNT")
    )
)

In [0]:
# Show the results
sales_summary.show()

+----------------+------------------+------------------+-----------+
|     PRODUCTLINE|       TOTAL_SALES|     AVERAGE_SALES|SALES_COUNT|
+----------------+------------------+------------------+-----------+
|     Motorcycles|        1166388.34|3523.8318429003025|        331|
|    Vintage Cars|1903150.8399999999|3135.3391103789127|        607|
|Trucks and Buses|        1127789.84|3746.8100996677745|        301|
|    Classic Cars|3919615.6599999983|4053.3771044467408|        967|
|          Planes| 975003.5699999997| 3186.286176470587|        306|
|           Ships| 714437.1299999997|3053.1501282051267|        234|
|          Trains|         226243.47| 2938.226883116883|         77|
+----------------+------------------+------------------+-----------+



In [0]:
# 2. PartitionBy Transformation: Partition Data by Year and Month for Staging
output_path = 'abfss://staging@dpstorage21.dfs.core.windows.net/sales_data_cleaned.csv'
(
    sales_data_df.repartition("YEAR_ID", "MONTH_ID")
    .write.mode("overwrite")
    .partitionBy("YEAR_ID", "MONTH_ID")
    .parquet(output_path)
)

In [0]:
print(f"Data partitioned by 'YEAR_ID' and 'MONTH_ID' and saved to {output_path}")

Data partitioned by 'YEAR_ID' and 'MONTH_ID' and saved to abfss://staging@dpstorage21.dfs.core.windows.net/sales_data_cleaned.csv
